In [2]:
import gmaps
import gmaps.datasets
import os

from lib import HowCloseIsItService

how_close_service = HowCloseIsItService()
biedry = how_close_service.get_all_of_ammenity("biedronka", dry_run=True)

ModuleNotFoundError: No module named 'sklearn'

In [3]:
df = gmaps.datasets.load_dataset_as_df('earthquakes')

In [6]:
locations = df[['latitude', 'longitude']]

In [7]:
gmaps.configure(api_key=os.getenv("GOOGLE_MAPS_API_KEY"))
fig = gmaps.figure()

fig.add_layer(gmaps.heatmap_layer(locations, df['magnitude']))

In [8]:
fig

Figure(layout=FigureLayout(height='420px'))